In [12]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from chatbot import ChatBot, load_api_key
from OktSentiment import Sentiment
from tarot import TarotBot

app = Flask(__name__)
CORS(app)

# ChatBot 인스턴스 생성
api_key = load_api_key()
if not api_key:
    raise ValueError("API key not found or invalid. Please check your api_key.txt file.")
chatbot = ChatBot(api_key, "C:\\Users\\SMHRD\\Desktop\\유형,대처,유형설명(마카님 삭제).csv")  # CSV 파일 경로를 적절히 수정하세요
#C:\Users\SMHRD\Documents\카카오톡 받은 파일\원인-대처종합.csv //c:/Users/SMHRD/원인-대처종합.csv
# Sentiment 인스턴스 생성
sentiment_analyzer = Sentiment()

@app.route("/chatbot", methods=["POST"])
def chat():
    data = request.json
    print("Received data:", data)
    user_nick = data.get("user_nick")
    user_input = data.get("input")
    
    if user_input:
        try:
            response = chatbot.process_input(user_nick, user_input)
            if response.get("isProblem"):
                # 형태소 분석 및 감정 분석 수행
                sentiment_results = sentiment_analyzer.parse([user_input])
                # 결과가 딕셔너리 형태로 바꾸고 소수점 소수점 둘쨰자리까지
                if 'polarity' in sentiment_results and len(sentiment_results['polarity']) > 0:
                    sentiment_results = sentiment_results['polarity'][0]
                    sentiment_results = {key: round(value * 100, 2) for key, value in sentiment_results.items()}

                response.update({
                    "sentiment": sentiment_results
                })

            return jsonify(response)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            return jsonify({"error": f"An error occurred while processing your request: {str(e)}"}), 500
    else:
        return jsonify({"error": "No input provided"}), 400

# 타로카드 라우트
@app.route("/api/interpret",methods=["POST"])

def tarot_card():
    data = request.json
    cards = data.get("cards")
    user_input = data.get("user_input")
    user_select = data.get("user_select")
    print("recived data: ",data)
    if not cards or len(cards) != 3:
        return jsonify({"error": "카드를 다시 선택해 주세요."}),400
    
    key_path = "c:/Users/SMHRD/api.txt"
    try:
        tarot = TarotBot(api_key_path=key_path)
        if user_input:
            print(user_input)
            response = tarot.interpret_cards(user_input,cards)
        else:
            response = tarot.general_reading(user_select,cards)
        print("response: ",response.get_json())
        return response

    except Exception as e:
        print("Error",str(e))
        return jsonify({"error":str(e)}),500

if __name__ == "__main__":
    app.run(port=7000)  

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit
127.0.0.1 - - [06/Sep/2024 15:28:41] "OPTIONS /api/interpret HTTP/1.1" 200 -


recived data:  {'user_mode': '일반', 'user_select': '오늘의 운세', 'cards': ['Four of Pentacles', 'The High Priestess', 'Justice'], 'user_input': None}


c:\Users\SMHRD\Desktop\MindRoad - 복사본\flask_modules\tarot.py:54: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  general_chain = LLMChain(llm=self.model, prompt=tarot_template_genal)
127.0.0.1 - - [06/Sep/2024 15:28:45] "POST /api/interpret HTTP/1.1" 200 -


response:  {'answer': {'cards': 'Four of Pentacles, The High Priestess, Justice', 'text': '- **Four of Pentacles**: 재정적 안정과 소유에 대한 집착\n- **The High Priestess**: 직관과 내면의 지혜\n- **Justice**: 공정성과 균형\n\n**오늘의 운세 종합 해석:**\n오늘은 재정적 안정이 중요한 날입니다. Four of Pentacles는 당신이 자신의 자원을 지키고자 하는 강한 욕구를 나타냅니다. 그러나 The High Priestess는 내면의 지혜와 직관을 활용하여 현재의 상황을 냉철하게 판단하라는 메시지를 전달합니다. Justice 카드는 공정성과 균형을 유지하는 것이 중요함을 강조합니다. 오늘은 재정적인 결정을 내릴 때 직관을 믿고, 공정하고 균형 잡힌 태도로 접근하세요. 이렇게 하면 안정과 만족을 얻을 수 있을 것입니다.', 'user_select': '오늘의 운세'}}


...

response:  {'answer': {'cards': 'King of Wands, Justice, The High Priestess', 'text': '**카드 의미 한줄 요약:**\n- **King of Wands**: 리더십과 결단력\n- **Justice**: 공정성과 균형\n- **The High Priestess**: 직관과 숨겨진 지식\n\n**금전 주제 종합 해석:**\n금전 문제에 있어서, 리더십과 결단력을 발휘해야 할 시기입니다. King of Wands는 당신이 주도적으로 상황을 이끌어 나가야 한다는 것을 의미합니다. Justice 카드는 모든 결정을 공정하고 균형 있게 내릴 필요가 있음을 상기시킵니다. 재정적인 판단을 할 때 감정에 치우치지 않고 객관적으로 접근하는 것이 중요합니다. The High Priestess는 직관과 내면의 지혜를 활용해 숨겨진 정보를 찾아내고 이를 바탕으로 현명한 결정을 내릴 것을 권장합니다. 따라서, 금전적인 문제를 해결하기 위해서는 리더십과 결단력을 가지고 공정성을 유지하면서도, 직관을 신뢰하는 것이 중요합니다.', 'user_select': '금전'}}


127.0.0.1 - - [06/Sep/2024 16:03:14] "OPTIONS /api/interpret HTTP/1.1" 200 -


recived data:  {'user_mode': '일반', 'user_select': '연애', 'cards': ['Five of Wands', 'Three of Pentacles', 'The Fool'], 'user_input': None}


127.0.0.1 - - [06/Sep/2024 16:03:17] "POST /api/interpret HTTP/1.1" 200 -


response:  {'answer': {'cards': 'Five of Wands, Three of Pentacles, The Fool', 'text': '**카드 의미 한줄 요약:**\n- **Five of Wands**: 충돌과 경쟁\n- **Three of Pentacles**: 협력과 팀워크\n- **The Fool**: 새로운 시작과 모험\n\n**연애 주제 종합 해석:**\n현재 당신의 연애 상황에서는 갈등이나 경쟁이 있을 수 있습니다(Five of Wands). 그러나 이러한 문제를 해결하기 위해서는 상호 협력과 팀워크가 중요합니다(Three of Pentacles). 새로운 시작을 두려워하지 말고, 용기 있게 모험을 떠나는 자세가 필요합니다(The Fool). 이러한 태도로 접근하면 관계가 긍정적으로 발전할 가능성이 큽니다. 새로운 기회를 받아들이고, 서로의 의견을 존중하며 협력하는 자세가 성공적인 연애를 이어나가는 열쇠입니다.', 'user_select': '연애'}}
